In [1]:
import pandas as pd
import folium
import requests
from folium import plugins

In [2]:
# The code was removed by Watson Studio for sharing.
CLIENT_ID = 'test' # your Foursquare ID
CLIENT_SECRET = 'test' # your Foursquare Secret
VERSION = 'test' # Foursquare API version

First a GET request for FourSquare API is constructed. Here, we find coffee near Helsinki center, in 3000 m radius. In general it's better to let requests to handle adding parameters to url than manually construct it.

In [72]:
request_parameters = {
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
    "v": '20200703',
    "section": "coffee",
    "near": "Mumbai",
    "radius": 3000,
    "limit": 50}

data = requests.get("https://api.foursquare.com/v2/venues/explore", params=request_parameters)

Parsing response to JSON. We get to following keys in our dictionary:

In [73]:
d = data.json()["response"]
d.keys()

dict_keys(['suggestedFilters', 'geocode', 'headerLocation', 'headerFullLocation', 'headerLocationGranularity', 'query', 'totalResults', 'suggestedBounds', 'groups'])

Most of the items on dictionary are giving some general information like the name of the city, it's geo code and so on. `groups` contains the actual search result.

In [74]:
d["headerLocationGranularity"], d["headerLocation"], d["headerFullLocation"]

('city', 'Mumbai', 'Mumbai')

In [75]:
d["suggestedBounds"], d["totalResults"]

({'ne': {'lat': 19.09860962158074, 'lng': 72.90742839551942},
  'sw': {'lat': 19.0493164536871, 'lng': 72.85939296430874}},
 59)

In [76]:
d["geocode"]

{'what': '',
 'where': 'mumbai',
 'center': {'lat': 19.07283, 'lng': 72.88261},
 'displayString': 'Mumbai, Mahārāshtra, India',
 'cc': 'IN',
 'geometry': {'bounds': {'ne': {'lat': 19.270308999589393,
    'lng': 72.97958690519049},
   'sw': {'lat': 18.89376311943323, 'lng': 72.7753930782989}}},
 'slug': 'mumbai',
 'longId': '72057594039203275'}

In groups we again have several different items, where type and name are irrelevant for given task. The full path to items is `data.json()["response"]["groups"][0]["items"]`.

Let's print couple of first items to get again better idea of the structure of the data:

In [77]:
d["groups"][0].keys()

dict_keys(['type', 'name', 'items'])

In [78]:
d["groups"][0]["type"], d["groups"][0]["name"]

('Recommended Places', 'recommended')

In [79]:
items = d["groups"][0]["items"]
print("number of items: %i" % len(items))
items[0]

number of items: 50


{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '4d9b39ed2ae860fc0f5a81cb',
  'name': 'Sofitel Mumbai BKC',
  'location': {'address': 'C 57 Bandra Kurla Complex, Bandra East, Maharashtra',
   'crossStreet': 'Bandra East',
   'lat': 19.0674478,
   'lng': 72.8690057,
   'labeledLatLngs': [{'label': 'display',
     'lat': 19.0674478,
     'lng': 72.8690057}],
   'postalCode': '400051',
   'cc': 'IN',
   'city': 'Mumbai',
   'state': 'Mahārāshtra',
   'country': 'India',
   'formattedAddress': ['C 57 Bandra Kurla Complex, Bandra East, Maharashtra (Bandra East)',
    'Mumbai 400051',
    'Mahārāshtra',
    'India']},
  'categories': [{'id': '4bf58dd8d48988d1fa931735',
    'name': 'Hotel',
    'pluralName': 'Hotels',
    'shortName': 'Hotel',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/hotel_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'

In [80]:
items[3]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '546f25c5498e2d4d056ad6eb',
  'name': 'Starbucks Coffee Capital',
  'location': {'address': 'The Capital',
   'lat': 19.063456936971313,
   'lng': 72.86157639300012,
   'labeledLatLngs': [{'label': 'display',
     'lat': 19.063456936971313,
     'lng': 72.86157639300012}],
   'cc': 'IN',
   'country': 'India',
   'formattedAddress': ['The Capital', 'India']},
  'categories': [{'id': '4bf58dd8d48988d1e0931735',
    'name': 'Coffee Shop',
    'pluralName': 'Coffee Shops',
    'shortName': 'Coffee Shop',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}},
 'referralId': 'e-5-546f25c5498e2d4d056ad6eb-3'}

Based on this, a simple for loop over the items is collecting some relevant data from the array, and after that DataFrame is created based on this parsed data.

In [81]:
df_raw = []
for item in items:
    venue = item["venue"]
    categories, uid, name, location = venue["categories"], venue["id"], venue["name"], venue["location"]
    assert len(categories) == 1
    shortname = categories[0]["shortName"]
#     address = location["address"]
    if not "postalCode" in location:
        continue
    postalcode = location["postalCode"]
    lat = location["lat"]
    lng = location["lng"]
    datarow = (uid, name, shortname, address, postalcode, lat, lng)
    df_raw.append(datarow)
df = pd.DataFrame(df_raw, columns=["uid", "name", "shortname", "address", "postalcode", "lat", "lng"])
print("found %i cafes" % len(df))
df.head()

found 25 cafes


,uid,name,shortname,address,postalcode,lat,lng
0,4d9b39ed2ae860fc0f5a81cb,Sofitel Mumbai BKC,Hotel,Oberoi Mall,400051,19.067448,72.869006
1,5286f821498e858fbcaef672,Starbucks Coffee: A Tata Alliance,Coffee Shop,Oberoi Mall,400051,19.069457,72.869375
2,548fd811498ed58c1ff94909,212 All Day,Café,Oberoi Mall,400070,19.086338,72.889073
3,4b0587cef964a5208aa222e3,Le Café,Café,Oberoi Mall,400 071,19.061791,72.899479
4,506c245be889751dc706f806,Mad Over Donuts,Donuts,Oberoi Mall,400086,19.087812,72.904909


Some density based estimator is giving a good tip where to start a new coffee business. There's a `HeatMap` plugin ready in Folium, let's use that, and visualize all the existing Cafes to same map:

In [82]:
mum_center = d["geocode"]["center"]
mum_center

{'lat': 19.07283, 'lng': 72.88261}

In [83]:
# create map of Mumbai using latitude and longitude values
map_mum = folium.Map(location=[mum_center["lat"], mum_center["lng"]], zoom_start=14)

def add_markers(df):
    for (j, row) in df.iterrows():
        label = folium.Popup(row["name"], parse_html=True)
        folium.CircleMarker(
            [row["lat"], row["lng"]],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_helsinki)

add_markers(df)
hm_data = df[["lat", "lng"]].to_numpy().tolist()
map_mum.add_child(plugins.HeatMap(hm_data))

map_mum

## Results

Based on these results, one possibly good location for new Cafe would be on the Santa Cruz Chembur Link Road.

In [84]:
lat = 19.07283
lng = 72.88261
map_it = folium.Map(location=[lat, lng], zoom_start=17)
add_markers(df)
folium.CircleMarker(
    [lat, lng],
    radius=15,
    popup="Our Cafe!",
    color='red',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,# create map of Mumbai using latitude and longitude values
map_mum = folium.Map(location=[mum_center["lat"], mum_center["lng"]], zoom_start=14)

def add_markers(df):
    for (j, row) in df.iterrows():
        label = folium.Popup(row["name"], parse_html=True)
        folium.CircleMarker(
            [row["lat"], row["lng"]],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_helsinki)

add_markers(df)
hm_data = df[["lat", "lng"]].to_numpy().tolist()
map_mum.add_child(plugins.HeatMap(hm_data))

map_mum
    parse_html=False).add_to(map_it)
map_it